In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import folium
import pymysql
import pandas as pd
from dash.exceptions import PreventUpdate

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the web app
app.layout = html.Div([
    html.H1("Railway Data Map"),
    html.Div([
        dcc.Dropdown(id="division-dropdown", options=[
            {'label': 'BB', 'value': 'BB'},
            {'label': 'JU', 'value': 'JU'},
            {'label': 'BSP', 'value': 'BSP'}
        ], value='BB'),
        dcc.Dropdown(id="direction-dropdown", options=[
            {'label': 'DN', 'value': 'DN'},
            {'label': 'UP', 'value': 'UP'}
        ], value='DN'),
        dcc.Input(id="date-input", type="text", placeholder="Enter Date (e.g., '2023-10-30')"),
        html.Button("Generate Map", id="generate-button"),
    ]),
    html.Iframe(id="railway-map", width="100%", height="600px"),
])

# Define a function to connect to your MySQL database
def connect_to_database():
    db = pymysql.connect(host='X', port=Y, user='Z', password='W', database='DEMO_DB')
    return db

# Define a function to fetch coordinates and speed data from the database based on user parameters
def get_data_from_db(date, direction, division):
    db = connect_to_database()
    cursor = db.cursor()
    query = "SELECT entrsttnlttd, entrsttnlgtd, AVG(speedinclstbl) AS avg_speed FROM speed_data WHERE `tdate` = %s AND `drtn` = %s AND `dvsn` = %s GROUP BY entrsttnlttd, entrsttnlgtd"
    cursor.execute(query, (date, direction, division))
    data = cursor.fetchall()
    db.close()
    return data

# Callback to generate the map
@app.callback(
    Output("railway-map", "srcDoc"),
    Input("generate-button", "n_clicks"),
    Input("division-dropdown", "value"),
    Input("direction-dropdown", "value"),
    Input("date-input", "value"),
)
def generate_map(n_clicks, division, direction, date):
    if n_clicks is None:
        raise PreventUpdate

    data = get_data_from_db(date, direction, division)

    # Create a map centered on an initial location
    m = folium.Map(location=[A, B], zoom_start=9)

    # Define categories and their corresponding colors
    categories = {
        "0-10": "blue",
        "10-20": "green",
        "20-30": "orange",
        "30+": "red"
    }

    styles = []

    for i in range(len(data) - 1):
        lat_str1 = data[i][0]
        lon_str1 = data[i][1]
        lat_str2 = data[i + 1][0]
        lon_str2 = data[i + 1][1]
        speed = float(data[i][2]) if data[i][2] is not None else 0

        if lat_str1 and lon_str1 and lat_str2 and lon_str2:
            lat1 = float(lat_str1)
            lon1 = float(lon_str1)
            lat2 = float(lat_str2)
            lon2 = float(lon_str2)

            if speed <= 10:
                category = "0-10"
            elif speed <= 20:
                category = "10-20"
            elif speed <= 30:
                category = "20-30"
            else:
                category = "30+"

            color = categories[category]

            style = {'color': color, 'weight': 3}

            coords = [(lat1, lon1), (lat2, lon2)]
            folium.PolyLine(locations=coords, **style).add_to(m)

    # Save the map to an HTML file and return it as an iframe
    map_filename = "plot.html"
    m.save(map_filename)

    with open(map_filename, "r") as file:
        src_doc = file.read()

    return src_doc

if __name__ == "__main__":
    app.run_server(debug=True)


C:\Users\varun\AppData\Local\Temp\ipykernel_11444\36852673.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\varun\AppData\Local\Temp\ipykernel_11444\36852673.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
